<a href="https://colab.research.google.com/github/zkibnksmh/MachineLearningTasks/blob/main/UASML/Exercise_07_PyTorch_Experiment_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Zaky Ibnu Kusumah
#1103204213
#TK4404

##**07. PyTorch Experiment Tracking**

##**1. Get various imports and helper functions**

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.3 MB/s eta 0:00:00
  ERROR: HTTP error 403 while getting https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from https://download.pytorch.org/whl/nightly/cu113/nvidia-cuda-nvrtc-cu12/)
ERROR: Could not install requirement nvidia-cuda-nvrtc-cu12==12.1.105 from https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch) because of HTTP error 403 Client Error: Forbidden for url: https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl for URL https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-ma

## Penjelasan Kode Pemeriksaan Versi PyTorch dan torchvision:

Kode tersebut digunakan untuk memeriksa versi PyTorch dan torchvision yang terpasang pada environment saat ini. Jika versi yang terpasang tidak memenuhi persyaratan (PyTorch 1.12+ dan torchvision 0.13+), maka kode akan menginstall versi nightly terbaru dari PyTorch dan torchvision.

**Langkah-langkah Kode:**

1. Coba impor modul `torch` dan `torchvision`.
2. Gunakan pernyataan `assert` untuk memastikan versi `torch` >= 1.12 dan `torchvision` >= 0.13. Jika tidak memenuhi, lempar pesan kesalahan.
3. Jika berhasil melewati pemeriksaan, tampilkan versi `torch` dan `torchvision` yang terpasang.
4. Jika ada kesalahan atau versi tidak memenuhi persyaratan, cetak pesan bahwa versi tidak sesuai dan instal versi nightly terbaru dari `PyTorch` dan `torchvision`.
5. Setelah instalasi selesai, impor kembali modul `torch` dan `torchvision` dan tampilkan versi yang telah terpasang.

Tujuan dari kode ini adalah memastikan bahwa notebook dapat berjalan dengan versi PyTorch dan torchvision yang memenuhi persyaratan untuk menjalankan kode dengan API yang diperbarui.


In [4]:
# torch.cuda.is_available(): Fungsi ini memeriksa apakah sistem memiliki GPU yang dapat digunakan menggunakan modul CUDA dari PyTorch. Jika tersedia, fungsi ini mengembalikan True, dan jika tidak, mengembalikan False.

# device = "cuda" if torch.cuda.is_available() else "cpu":
# Pada baris ini, menggunakan ekspresi kondisional (ternary operator) untuk menetapkan nilai variabel device.
# Jika torch.cuda.is_available() bernilai True, maka device diatur ke "cuda" (menggunakan GPU), dan jika False, diatur ke "cpu" (menggunakan CPU).

# device:
# Baris ini mencetak nilai variabel device, yang dapat berupa "cuda" jika GPU tersedia atau "cpu" jika tidak.
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [5]:
# Get regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4036, done.
remote: Counting objects: 100% (1224/1224), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 4036 (delta 1068), reused 1078 (delta 996), pack-reused 2812
Receiving objects: 100% (4036/4036), 651.02 MiB | 30.14 MiB/s, done.
Resolving deltas: 100% (2361/2361), done.
Updating files: 100% (248/248), done.


In [6]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [7]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.

    Returns:
        pathlib.Path to downloaded data.

    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it...
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...")
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)

    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizza_steak_sushi')

In [8]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None):
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)


# Create a test writer
writer = create_writer(experiment_name="test_experiment_name",
                       model_name="this_is_the_model_name",
                       extra="add_a_little_extra_if_you_want")

[INFO] Created SummaryWriter, saving to: runs/2024-01-03/test_experiment_name/this_is_the_model_name/add_a_little_extra_if_you_want...


Fungsi `create_writer` digunakan untuk membuat instance dari `SummaryWriter` dari modul TensorBoard PyTorch. Writer ini akan menyimpan log ke direktori tertentu yang dibentuk dari kombinasi `runs/timestamp/experiment_name/model_name/extra`. Timestamp adalah tanggal saat ini dalam format YYYY-MM-DD. Contoh penggunaan fungsi ini diberikan pada bagian komentar dalam blok dokumen tersebut.

In [9]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

# Add writer parameter to train()
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
      For example if training for epochs=2:
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

Fungsi `train` digunakan untuk melatih dan menguji model PyTorch. Fungsi ini menggunakan metode `train_step()` dan `test_step()` untuk menjalankan model PyTorch melalui beberapa epoch, melatih dan menguji model dalam satu loop epoch yang sama. Fungsi juga menghitung, mencetak, dan menyimpan metrik evaluasi selama proses.

Fungsi ini menerima parameter baru, yaitu `writer`, yang merupakan instance dari `SummaryWriter` dari modul TensorBoard PyTorch. Parameter ini digunakan untuk menyimpan hasil log ke direktori tertentu jika diberikan.

Hasil dari proses pelatihan dan pengujian disimpan dalam dictionary `results`, yang berisi loss dan akurasi untuk pelatihan dan pengujian setiap epoch. Dictionary ini memiliki format:

```python
{
    "train_loss": [...],
    "train_acc": [...],
    "test_loss": [...],
    "test_acc": [...]
}


##**Download data**

In [10]:
# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data/pizza_steak_sushi directory exists, skipping download.
[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


Kode di atas digunakan untuk mengunduh data latih dengan persentase 10% dan 20% jika data tersebut belum ada di sistem. Fungsi `download_data` dipanggil dua kali, masing-masing untuk mengunduh dataset dengan persentase 10% dan 20%.

- `data_10_percent_path`: Variabel ini menyimpan hasil panggilan fungsi `download_data` untuk mendapatkan data latih dengan persentase 10%.
- `data_20_percent_path`: Variabel ini menyimpan hasil panggilan fungsi `download_data` untuk mendapatkan data latih dengan persentase 20%..

Fungsi `download_data` sendiri bertanggung jawab untuk mengunduh dataset dari URL sumber dan menyimpannya di direktori tujuan yang ditentukan.


In [11]:
# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_10_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: data/pizza_steak_sushi/train
Training directory 20%: data/pizza_steak_sushi_20_percent/train
Testing directory: data/pizza_steak_sushi/test


Kode di atas digunakan untuk menyiapkan jalur direktori dataset latih dan uji dengan persentase 10% dan 20%. Proses ini melibatkan pengaturan jalur untuk direktori latih dan uji menggunakan jalur yang diunduh sebelumnya.

- `train_dir_10_percent`: Variabel ini menentukan jalur direktori latih untuk dataset dengan persentase 10%. Jalur ini dibentuk dengan menggabungkan jalur data_10_percent_path dengan subdirektori "train". Hasilnya adalah jalur ke direktori yang berisi dataset latih dengan persentase 10%.

- `train_dir_20_percent`: Variabel ini menentukan jalur direktori latih untuk dataset dengan persentase 20%. Jalur ini dibentuk dengan menggabungkan jalur data_20_percent_path dengan subdirektori "train". Hasilnya adalah jalur ke direktori yang berisi dataset latih dengan persentase 20%.

- `test_dir`: Variabel ini menentukan jalur direktori uji yang akan digunakan untuk kedua dataset dengan persentase 10% dan 20%. Jalur ini dibentuk dengan menggabungkan jalur data_10_percent_path dengan subdirektori "test". Hasilnya adalah jalur ke direktori yang berisi dataset uji yang sama untuk kedua dataset latih.

Selanjutnya, kode mencetak jalur direktori latih dengan persentase 10%, jalur direktori latih dengan persentase 20%, dan jalur direktori uji.

In [12]:
from torchvision import transforms

# Create a transform to normalize data distribution to be inline with ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
                                 std=[0.229, 0.224, 0.225])

# Create a transform pipeline
simple_transform = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ToTensor(), # get image values between 0 & 1
                                       normalize
])

Kode di atas digunakan untuk membuat transformasi pada data gambar menggunakan modul `transforms` dari PyTorch. Transformasi ini biasanya diterapkan pada dataset sebelum dimasukkan ke dalam model untuk pelatihan atau inferensi. Penjelasan dari kode tersebut sebagai berikut:

- `normalize`: Variabel ini menggunakan transformasi normalisasi untuk mengubah distribusi warna gambar agar sejajar dengan ImageNet. Nilai rata-rata dan standar deviasi yang digunakan diatur sesuai dengan nilai yang telah diukur dari ImageNet dataset. Transformasi ini membantu model untuk belajar lebih baik dengan mendapatkan gambar yang memiliki distribusi warna yang serupa dengan dataset ImageNet.

- `simple_transform`: Variabel ini menggunakan modul `Compose` untuk membuat sebuah pipeline transformasi yang terdiri dari beberapa tahap transformasi. Transformasi yang diterapkan melibatkan:
  - `Resize((224, 224))`: Mengubah ukuran gambar menjadi 224x224 piksel.
  - `ToTensor()`: Mengubah nilai piksel gambar menjadi rentang antara 0 dan 1 dan mengonversinya menjadi tensor.
  - `normalize`: Melakukan normalisasi distribusi warna gambar sesuai dengan nilai mean dan std yang telah diatur sebelumnya.

Transformasi ini bertujuan untuk mempersiapkan data gambar sehingga sesuai untuk digunakan sebagai input pada model yang telah dilatih dengan data ImageNet.


##**Turn data into DataLoaders**

In [13]:
BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Create 20% training and test DataLoaders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 8 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


Kode di atas digunakan untuk membuat DataLoaders dari dataset pelatihan dan pengujian dengan menggunakan fungsi `create_dataloaders` dari modul `data_setup`. Penjelasan dari kode tersebut sebagai berikut:

- `BATCH_SIZE = 32`: Variabel ini menentukan ukuran batch yang akan digunakan dalam setiap iterasi pelatihan.

- `train_dataloader_10_percent`: DataLoader yang dibuat menggunakan dataset pelatihan 10%. Menggunakan fungsi `create_dataloaders`, dengan parameter berupa direktori pelatihan 10%, direktori pengujian, transformasi data (`simple_transform`), dan ukuran batch (`BATCH_SIZE`). DataLoader ini akan digunakan untuk melatih model pada 10% dari dataset.

- `train_dataloader_20_percent`: DataLoader yang dibuat menggunakan dataset pelatihan 20%. Sama seperti sebelumnya, namun kali ini menggunakan dataset pelatihan 20%.

- `test_dataloader`: DataLoader yang digunakan untuk pengujian. Sama dengan DataLoader pengujian pada eksperimen sebelumnya, ini digunakan untuk evaluasi performa model pada dataset pengujian yang sama pada kedua eksperimen.

- `class_names`: Variabel yang menyimpan nama kelas/kategori dari dataset. Dihasilkan oleh fungsi `create_dataloaders`.

- Output yang dicetak:
  - Jumlah batch dalam dataset pelatihan 10%.
  - Jumlah batch dalam dataset pelatihan 20%.
  - Jumlah batch dalam dataset pengujian (digunakan oleh kedua eksperimen).
  - Jumlah kelas dan nama kelas dalam dataset.
